In [1]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.preprocessing import StandardScaler

import tqdm.notebook as tq
import warnings
warnings.filterwarnings('ignore')

import utils_cl
%matplotlib notebook
%load_ext autoreload

In [2]:
full_df = pd.read_csv("dataset/residual_weekly_consumption.csv")
full_df.head(3)

,meter_id,START_WEEKDAY,T2,T3,T4,T5,T6,T7,CN
0,PP01000100340001,2017-01-02,0.321275,0.040023,-0.185360,-0.097297,0.104045,-0.170828,-0.082016
1,PP01000100340001,2017-01-09,0.032041,0.195984,-0.085221,-0.029924,-0.038465,0.257508,-0.087102
2,PP01000100340001,2017-01-16,-0.121479,-0.100963,0.163844,-0.019069,-0.015114,-0.049732,0.037160


In [4]:
list_fraud = ['PP03000806717001',
 'PP01000624985001',
 'PP09000811262001',
 'PP07000600209001',
 'PP05000515674001',
 'PP07000613618001',
 'PP05000944240001',
 'PP07000683750001',
 'PP05000501075001',
 'PP01000101975001',
 'PP05000504291001',
 'PP05000517007001',
 'PP05000524362001',
 'PP09000907245001',
 'PP01000103454001',
 'PP05000515078001',
 'PP01000621525001',
 'PP07000668984001',
 'PP03000804137001',
 'PP09000802199001',
 'PP03000809863001',
 'PP05000960339001',
 'PP03000810040001',
 'PP03000804097001',
 'PP07000693227001',
 'PP05000524904001',
 'PP05000951373001',
 'PP05000989390001',
 'PP05000961578001',
 'PP07000613734001',
 'PP09000892588001',
 'PP01000101145001',
 'PP01000117448001',
 'PP09000810551001',
 'PP09000837585001',
 'PP05000968892001',
 'PP09000890287001',
 'PP09000808816001',
 'PP03000832565001',
 'PP09000821975001',
 'PP05000509756001',
 'PP05000967753001',
 'PP05000941329001',
 'PP05000953931001',
 'PP09000817228001',
 'PP03000898466001',
 'PP05000522325001',
 'PP07000600208001',
 'PP05000948327001',
 'PP01000140094001',
 'PP05000501411001',
 'PP07000673219001',
 'PP07000682085001',
 'PP01000131431001',
 'PP07000700839001',
 'PP07000713473001',
 'PP05000977900001',
 'PP07000678169001',
 'PP03000805922001',
 'PP09000808752001',
 'PP09000120555001',
 'PP05000502843001',
 'PP05000516894001',
 'PP07000683751001',
 'PP07000684209001',
 'PP05000967512001']

## Testing PCA

In [40]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
X_principal = pca.fit_transform(full_df.iloc[:, 2:])
X_principal = pddd.DataFrame(X_principal)

## Testing Model

In [29]:
X_principal = full_df.iloc[:, 2:]

In [41]:
def Scheme_used():
    
    X_normalized = X_principal
    df2 = full_df
    
    from sklearn.cluster import KMeans
    distance_used = utils_cl.distance_ED
    cluster_number  = 3
    n_iters = 750
    km = KMeans(
        n_clusters=cluster_number, init='random',
        n_init=10, max_iter=n_iters,
        tol=1e-04, random_state=200
    )

    y_km = km.fit_predict(X_normalized)
    centroids = km.cluster_centers_
    
    dict_DC = {}
    for i in range(cluster_number):
        dict_DC[i+1] = []
    for i, j in tq.tqdm_notebook(enumerate(y_km), total = len(y_km)):        
        dict_DC[j+1].append(distance_used(centroids[j], X_normalized.iloc[i, :])) 
    quantile_radius = []
    percentile = 0.95
    for j in range(cluster_number):
        quantile_radius.append(pd.DataFrame(dict_DC[j+1]).quantile(percentile, axis = 0)[0])
    def test_fraud_cluster(X_sample, cluster_belong, radius_cent, thresh_KM):
        dist2cent = distance_used(X_sample, centroids[cluster_belong - 1])
        is_fraud = (dist2cent > radius_cent[cluster_belong-1] * thresh_KM)
        return is_fraud
    def prediction_result(test_set, radius_cent, thresh_KM = 1.0):
        N = test_set.shape[0]
        mdd_fraud_1week = []
        Y_predict = []
        ind_fraud_predicted = []
        count_fraud_predicted = [0]*cluster_number
        for i in tq.tqdm_notebook(range(N)):
            X_sample = test_set.iloc[i, :]
            cluster_belong = y_km[i] + 1
            test_result = test_fraud_cluster(X_sample, cluster_belong, radius_cent, thresh_KM)
            if test_result == True :
                count_fraud_predicted[cluster_belong-1] += 1
                mdd_fraud_1week.append(df2.iloc[i, 0])
                ind_fraud_predicted.append(i)
            Y_predict.append(test_result)
        return list(set(mdd_fraud_1week)), Y_predict, ind_fraud_predicted,count_fraud_predicted
    
    test_set = X_normalized
    radius_cent = quantile_radius
    thresh_KM = 1
    mdd_fraud_1week, Y_predict, ind_fraud_predicted, count_fraud_predicted = prediction_result(test_set, radius_cent, thresh_KM)
    test_df = pd.DataFrame()
    test_df['meter_id'] = df2.meter_id
    test_df['PRED'] = Y_predict
    test_df = test_df.groupby(['meter_id'])['PRED'].apply(list)
    
    consecutive_week = 2
    def test_consecutive_week(l):
        n = len(l)
        for i in range(n) :
            if l[i] == True:
                if sum(l[i : i+consecutive_week]) == consecutive_week:
                    return True
            else :
                continue
        return False
    mdd_fraud_n_week = []
    for i in range(test_df.shape[0]):
        l = test_df[i]
        if test_consecutive_week(l):
            mdd_fraud_n_week.append(test_df.index[i])
            
    n_fraud = len(set(mdd_fraud_n_week) & set(list_fraud))
    res_1 = n_fraud/len(list_fraud)
    res_2 = (len(mdd_fraud_n_week) - n_fraud)/1000
    return res_1, res_2

## Testing Model

In [42]:
def Scheme2():
    X_normalized = full_df.iloc[:, 2:]
    df2 = full_df
    
    from sklearn.cluster import KMeans
    distance_used = utils_cl.distance_ED
    cluster_number  = 3
    n_iters = 750
    km = KMeans(
        n_clusters=cluster_number, init='random',
        n_init=10, max_iter=n_iters,
        tol=1e-04, random_state=200
    )

    y_km = km.fit_predict(X_normalized)
    centroids = km.cluster_centers_
    
    dict_DC = {}
    for i in range(cluster_number):
        dict_DC[i+1] = []
    for i, j in tq.tqdm_notebook(enumerate(y_km), total = len(y_km)):        
        dict_DC[j+1].append(distance_used(centroids[j], X_normalized.iloc[i, :])) 
    quantile_radius = []
    percentile = 0.95
    for j in range(cluster_number):
        quantile_radius.append(pd.DataFrame(dict_DC[j+1]).quantile(percentile, axis = 0)[0])
    def test_fraud_cluster(X_sample, cluster_belong, radius_cent, thresh_KM):
        dist2cent = distance_used(X_sample, centroids[cluster_belong - 1])
        is_fraud = (dist2cent > radius_cent[cluster_belong-1] * thresh_KM)
        return is_fraud
    def prediction_result(test_set, radius_cent, thresh_KM = 1.0):
        N = test_set.shape[0]
        mdd_fraud_1week = []
        Y_predict = []
        ind_fraud_predicted = []
        count_fraud_predicted = [0]*cluster_number
        for i in tq.tqdm_notebook(range(N)):
            X_sample = test_set.iloc[i, :]
            cluster_belong = y_km[i] + 1
            test_result = test_fraud_cluster(X_sample, cluster_belong, radius_cent, thresh_KM)
            if test_result == True :
                count_fraud_predicted[cluster_belong-1] += 1
                mdd_fraud_1week.append(df2.iloc[i, 0])
                ind_fraud_predicted.append(i)
            Y_predict.append(test_result)
        return list(set(mdd_fraud_1week)), Y_predict, ind_fraud_predicted,count_fraud_predicted
    
    test_set = X_normalized
    radius_cent = quantile_radius
    thresh_KM = 1
    mdd_fraud_1week, Y_predict, ind_fraud_predicted, count_fraud_predicted = prediction_result(test_set, radius_cent, thresh_KM)
    test_df = pd.DataFrame()
    test_df['meter_id'] = df2.meter_id
    test_df['PRED'] = Y_predict
    test_df = test_df.groupby(['meter_id'])['PRED'].apply(list)
    
    consecutive_week = 2
    def test_consecutive_week(l):
        n = len(l)
        for i in range(n) :
            if l[i] == True:
                if sum(l[i : i+consecutive_week]) == consecutive_week:
                    return True
            else :
                continue
        return False
    mdd_fraud_n_week = []
    for i in range(test_df.shape[0]):
        l = test_df[i]
        if test_consecutive_week(l):
            mdd_fraud_n_week.append(test_df.index[i])
            
    n_fraud = len(set(mdd_fraud_n_week) & set(list_fraud))
    res_1 = n_fraud/len(list_fraud)
    res_2 = (len(mdd_fraud_n_week) - n_fraud)/1000
    return res_1, res_2

(0.45454545454545453, 0.202)

In [ ]:
# Clustering
X_normalized = full_df.iloc[:, 2:]

from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
X_principal = pca.fit_transform(full_df.iloc[:, 2:])
X_principal = pd.DataFrame(X_principal)

from sklearn.cluster import DBSCAN
# DBSCAN
eps = 0.2
min_samples = 10
labels_1 = utils_cl._DBSCAN(X_principal, eps, min_samples)

In [58]:
X_normalized.shape

(66540, 7)

In [ ]:
from collections import Counter
Counter(labels_1)